# 聊天对话
我们已经做了很多关于大模型的实验，每一步都带来我们对于提示词新的认知，事实上，你可以通过提示词工程进行更有趣的实验，比如指导大语言模型系统如何表现，指定它的行为意图和身份。 当你在构建对话系统，如客户服务聊天机器人时，这尤其有用。

下例中，你可以独立创建一个对话系统，该系统能够基于问题给出技术性和科学的回答。 你可以关注我们是如何通过指令明确地告诉模型应该如何表现。 这种应用场景有时也被称为角色提示（Role Prompting）。

但是需要注意的是，聊天往往意味着需要对话的历史记录，我们需要将历史记录添加到提示词中，不过这并不难，还记得我们在推理代码中，构建的对话信息message吗？我们只需要将历史信息根据对话模板添加到提示词中就可以做到。

In [1]:
### 加载模型Qwen模型
from modelscope import AutoModelForCausalLM, AutoTokenizer

model_name_or_path_instruct = '/home/lixinyu/weights/Qwen2.5-3B-Instruct'  # 替换为你下载的模型路径
tokenizer_instruct = AutoTokenizer.from_pretrained(model_name_or_path_instruct)
model_instruct = AutoModelForCausalLM.from_pretrained(model_name_or_path_instruct,device_map='cuda:0', torch_dtype='auto')

/home/lixinyu/miniconda3/envs/prompt/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/home/lixinyu/miniconda3/envs/prompt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.55it/s]


In [13]:
from typing import List, Tuple, Dict, Any

# ---------- 单轮推理 ----------
def inference(
        model,
        tokenizer,
        history: List[Tuple[str, str]],   # [(user0, asst0), (user1, asst1), ...]
        user_utterance: str,
        max_new_tokens: int = 512,
        temperature: float = 0.9
) -> Dict[str, Any]:
    """
    根据当前历史 + 当前用户输入，生成模型回答，并返回：
    {
        "response": str,
        "generated_tokens_count": int,
        "history": List[Tuple[str, str]]   # 已追加本轮对话
    }
    """
    # 1) 构造 messages
    messages = [{"role": "system", "content": "你是专业的AI助手，请尽量回答简洁"}]
    for u, a in history:
        messages.append({"role": "user", "content": u})
        messages.append({"role": "assistant", "content": a})
    messages.append({"role": "user", "content": user_utterance})

    # 2) 模板化 & tokenize
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    # 3) 生成
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        do_sample=True,
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in
        zip(model_inputs.input_ids, generated_ids)
    ]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    generated_tokens_count = len(generated_ids[0])

    # 4) 更新历史
    new_history = history + [(user_utterance, response)]

    return {
        "response": response,
        "generated_tokens_count": generated_tokens_count,
        "history": new_history
    }

# ---------- 方便的多轮封装 ----------
def chat_turn(
        model,
        tokenizer,
        history: List[Tuple[str, str]],
        user_utterance: str,
        **gen_kwargs
) -> List[Tuple[str, str]]:
    """
    最简洁用法：输入历史 + 用户一句，直接返回更新后的历史。
    """
    res = inference(model, tokenizer, history, user_utterance, **gen_kwargs)
    return res

In [16]:
history=[]

prompt_1="我想学做一道最简单的家常菜"
response=chat_turn(model_instruct,tokenizer_instruct,history,prompt_1)
history=response['history']
print(response['response']+'\n')

prompt_2="能再具体点吗？比如火候。"
response=chat_turn(model_instruct,tokenizer_instruct,history,prompt_2)
history=response['history']
print(response['response']+'\n')

prompt_3="我不吃糖，怎么办？"
response=chat_turn(model_instruct,tokenizer_instruct,history,prompt_3)
history=response['history']
print(response['response']+'\n')

你可以尝试做西红柿炒鸡蛋，做法简单：1. 鸡蛋打散；2. 西红柿切块；3. 热油锅炒鸡蛋至熟；4. 锅中再放油，炒西红柿至出汁；5. 把鸡蛋倒进西红柿中，快速翻炒均匀即可。

当然可以：

1. **准备材料**：两个鸡蛋，一个中等大小的西红柿，少许盐和葱花（可选）。
2. **处理食材**：将鸡蛋打入碗中搅拌均匀；西红柿洗净后切成小块。
3. **烹饪步骤**：
   - 烧热锅后加入适量油，待油温六七成热时倒入鸡蛋液，待鸡蛋凝固变白后盛出备用。
   - 锅中留少量油，放入西红柿块，中小火翻炒至出汁。
   - 加入之前炒好的鸡蛋，撒入适量盐调味。
   - 快速翻炒均匀，使鸡蛋和西红柿充分混合。
   - 最后撒上葱花（如果使用），快速翻炒几下即可出锅。

4. **火候**：鸡蛋炒制时火要稍大一些，以快速定型并使其表面金黄。西红柿炒制时则要小火慢炒，以保持西红柿的鲜甜味和水分。

这样操作，一道美味的西红柿炒鸡蛋就完成了。

如果你不喜欢加糖，可以调整西红柿炒鸡蛋的做法，去掉调味品直接烹饪，这样做出来的味道也会很鲜美。具体步骤如下：

1. **准备材料**：两个鸡蛋，一个中等大小的西红柿，少许盐。
2. **处理食材**：将鸡蛋打入碗中搅拌均匀；西红柿洗净后切成小块。
3. **烹饪步骤**：
   - 烧热锅后加入适量油，待油温六七成热时倒入鸡蛋液，待鸡蛋凝固变白后盛出备用。
   - 锅中留少量油，放入西红柿块，加入少许盐，中小火翻炒至软烂出汁。
   - 加入之前炒好的鸡蛋，继续翻炒均匀。
   - 快速翻炒几下，让所有食材充分融合。

这样做的西红柿炒鸡蛋，味道依旧鲜美，不需要额外加糖。



需要注意的是，无论历史对话进行了多少轮，你永远都是提问的一方，模型永远都是回答的一方，因此，我们需要确保最终是user的内容。同时历史对话并不会无限长，模型总有输入的极限，一旦超过这个极限，模型总会记得最新的回答，事实上，我们也可以称为“滑动窗口”，这是模型对记忆的保存机制。

我们可以查看多轮对话在输入到模型中是什么样子：

In [17]:
system = "你是专业的AI助手，请尽量回答简洁"
user1="我想学做一道最简单的家常菜"
assistant1="你可以尝试做西红柿炒鸡蛋，做法简单：1. 鸡蛋打散；2. 西红柿切块；3. 热油锅炒鸡蛋至熟；4. 锅中再放油，炒西红柿至出汁；5. 把鸡蛋倒进西红柿中，快速翻炒均匀即可。"

user2="能再具体点吗？比如火候。"

messages = [
    {"role": "system", "content": system},
    {"role": "user", "content": user1},
    {"role": "assistant", "content": assistant1},
    {"role": "user", "content": user2}
]

# 运行下面的代码，我们将知道输入到模型中的是什么样子的文本
text = tokenizer_instruct.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
print(text)

<|im_start|>system
你是专业的AI助手，请尽量回答简洁<|im_end|>
<|im_start|>user
我想学做一道最简单的家常菜<|im_end|>
<|im_start|>assistant
你可以尝试做西红柿炒鸡蛋，做法简单：1. 鸡蛋打散；2. 西红柿切块；3. 热油锅炒鸡蛋至熟；4. 锅中再放油，炒西红柿至出汁；5. 把鸡蛋倒进西红柿中，快速翻炒均匀即可。<|im_end|>
<|im_start|>user
能再具体点吗？比如火候。<|im_end|>
<|im_start|>assistant



从结果上看，我们真的将历史对话给到了模型